In [2]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from market_analysis.heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta
from market_analysis.heystox_intraday.intraday_fetchdata import *
from market_analysis.tasks.stock_data_import_tasks import *
from market_analysis.tasks.day_trading_tasks import *
from django.core.cache import cache, caches
from datetime import time, timedelta, datetime
from market_analysis.heystox_intraday.trading_indicator import *
import time

In [15]:
sorted_stocks = SortedStocksList.objects.filter(created_at__date=datetime.now().date())
sorted_stock = SortedStocksList.objects.get(symbol__symbol="zeel", created_at__date=datetime.now().date())
print(sorted_stock.created_at)
print(sorted_stock.entry_type)

2020-02-10 10:23:16.448391
SELL


In [17]:
# MACD Crossover
today_date = datetime.today().date()
df = get_macd_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df.loc[df["date"] > str(today_date)]
new_df = df.copy(deep=True).drop(df.tail(1).index)
try:
    if sorted_stock.entry_type == "SELL":
        last_crossover = new_df[new_df.signal.str.endswith("SELL_CROSSOVER")].iloc[-1]
    elif sorted_stock.entry_type == "BUY":
        last_crossover = new_df[new_df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
except:
    last_crossover = None
if last_crossover is not None:
    df_after_last_crossover = df.loc[df["date"] > last_crossover.date]
    try:
        if last_crossover.signal == "SELL_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.macd_diff <= -0.070)].iloc[0]
        elif last_crossover.signal == "BUY_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.macd_diff >= 0.070)].iloc[0]
    except:
        crossover_signal = None
    if crossover_signal is not None:
        print("raju")
#         print(crossover_signal.item)
df.loc[(df.macd_diff <= -0.070)] # Signal
# df[df.signal.str.endswith("SELL_CROSSOVER")] # Crossover
last_crossover

raju


candle_type                             M5
open_price                           249.2
high_price                           249.4
low_price                           242.55
close_price                            243
volume                              974172
total_buy_quantity                    None
total_sell_quantity                   None
date                   2020-02-10 09:15:00
macd                              0.469266
macd_signal                       0.809343
macd_diff                        -0.340077
percentage                       -0.001596
signal                      SELL_CROSSOVER
Name: 75, dtype: object

In [20]:
# Shochastic Crossover
today_date = datetime.today().date()
df = get_stochastic_data(sorted_stock.symbol)
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "BUY"), "signal"] = "BUY_CROSSOVER"
df.loc[(df["signal"] != df["signal"].shift()) & (df["signal"] == "SELL"), "signal"] = "SELL_CROSSOVER"
df = df.loc[df["date"] > str(today_date)]
try:
    if sorted_stock.entry_type == "SELL":
        last_crossover = df[df.signal.str.endswith("SELL_CROSSOVER")].iloc[-6]
    elif sorted_stock.entry_type == "BUY":
        last_crossover = df[df.signal.str.endswith("BUY_CROSSOVER")].iloc[-1]
except:
    last_crossover = None
if last_crossover is not None:
    df_after_last_crossover = df.loc[df["date"] > last_crossover.date]
    try:
        if last_crossover.signal == "SELL_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.stoch_diff <= -22.70)].iloc[0]
        elif last_crossover.signal == "BUY_CROSSOVER":
            crossover_signal = df_after_last_crossover.loc[(df.stoch_diff >= 0.070)].iloc[0]
    except:
        crossover_signal = None
    if crossover_signal is not None and last_crossover is not None:
        print("RAJU")
# df.loc[(df.stoch_diff <= -22.70)] # Signal
df[df.signal.str.endswith("SELL_CROSSOVER")] # Crossover

,candle_type,open_price,high_price,low_price,close_price,volume,total_buy_quantity,total_sell_quantity,date,stoch,stoch_signal,stoch_diff,percentage,signal
79,M5,243.90,244.00,242.55,242.60,221892,None,None,2020-02-10 09:35:00,12.060302,16.582915,-4.522613,-0.545441,SELL_CROSSOVER
84,M5,238.80,238.80,237.25,238.10,265314,None,None,2020-02-10 10:00:00,10.169492,10.935111,-0.765620,-0.077860,SELL_CROSSOVER
87,M5,238.50,238.55,236.10,236.90,439294,None,None,2020-02-10 10:15:00,5.555556,10.037004,-4.481448,-0.248969,SELL_CROSSOVER
90,M5,236.65,236.75,234.00,234.35,831654,None,None,2020-02-10 10:30:00,3.500000,6.125266,-2.625266,-0.091884,SELL_CROSSOVER


In [5]:
# raju = ["adanient", "bhel", "dlf", "gmrinfra", "gail", "infratel", "ioc", "idea", "nhpc", "pnb", "relinfra", "recltd", "spencers", "tatapower", "vakrangee", "yesbank", "ujjivansfb", "granules", "mothersumi", "adanipower", "dishtv", "sail", "nationalum", "jswsteel", "federalbnk", "jindalstel", "idfcfirstb", "rvnl", "equitas"]
# fifth = date(2020,1,25)
# for r in raju:
#     symbol = Symbol.objects.get(symbol=r)
#     SortedStocksList.objects.create(symbol=symbol, entry_type="BUY")
# # Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju)
# for candle in Candle.objects.filter(date__date=fifth, symbol__symbol__in=raju):
#     candle.date = candle.date + timedelta(1)
#     candle.save()

In [11]:
import requests
from bs4 import BeautifulSoup
user = UserProfile.objects.get(user__email="sonupal129@gmail.com")
payload = {"username": 308687, "password": "Sonupal!@9", "password2fa": 1990, "TOKEN":"fbe2c9352ed86c06c20f003bee0babee7d6831a413026b670a907bf27789a4ff"}
headers= {"Host": "www1.nseindia.com", 
            "Cache-Control": "max-age=0",
            "Upgrade-Insecure-Requests": "1", "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
             "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
             }
# print(response.text.encode('utf8'))
# url_request.text.encode("utf-8")
response = requests.post(user.get_authentication_url(), data=payload)
response.content

b'{"message":"When Content-Type:application/x-www-form-urlencoded, URL cannot include query-string parameters (after \'?\'): \'/index/dialog/authorize?apiKey=VpcuCaUDK91y9JPy358T19nUWtq0UF6J8Yl3qVmr&redirect_uri=http%3A%2F%2F127.0.0.1%3A8000%2F&response_type=code\'"}'

In [7]:
# from django.conf import settings
# def import_daily_losers_gainers():
#     urls = {
#             "BUY": ["https://www1.nseindia.com/live_market/dynaContent/live_analysis/gainers/niftyGainers1.json",
#                     "https://www1.nseindia.com/live_market/dynaContent/live_analysis/gainers/jrNiftyGainers1.json"],
#             "SELL": ["https://www1.nseindia.com/live_market/dynaContent/live_analysis/losers/niftyLosers1.json",
#                     "https://www1.nseindia.com/live_market/dynaContent/live_analysis/losers/jrNiftyLosers1.json"]
#         }
    
#     nifty_movement = Symbol.objects.get(symbol="nifty_50").get_nifty_movement()
    
#     def response_filter(obj):
#         open_price = obj.get("openPrice")
#         change = float(obj.get("netPrice"))
#         if "," in open_price:
#             open_price = open_price.replace(",","")
#         open_price = float(open_price)
#         if open_price >= 100 and open_price <= 300 and change >= 1.2:
#             return obj
        
    
#     if nifty_movement in ("BUY", "SELL"):
#         import_urls = urls.get(nifty_movement)
#         if import_urls:
#             for url in import_urls:
#                 response = requests.get(url, headers=settings.NSE_HEADERS)
#                 if response.status_code == 200:
#                     responses = filter(response_filter, response.json().get("data"))
#                     for symbol in responses:
#                         try:
#                             stock = Symbol.objects.get(symbol=symbol.get("symbol").lower())
#                         except:
#                             stock = None
#                         if stock:
#                             SortedStocksList.objects.get_or_create(symbol=stock, entry_type=nifty_movement, created_at__date=datetime.now().date())
#                     return f"Data imported successfully! from {url}"
#                 return slack_message_sender.delay(channel="#random", text=f"Incorrect Url: {url}")
#             return "All Urls Data Imported Succefully"